In [1]:
from ccip_terminal import estimate_gas_limit_from_recent_ccip_dev, load_accounts, get_dynamic_gas_fees, ROUTER_MAP, send_ccip_transfer, estimate_dynamic_gas, CHAIN_MAP

In [2]:
from dotenv import load_dotenv
import os

# Gas Estimates

In [3]:
source_chain = 'optimism'

In [4]:
PRIVATE_KEY = os.getenv('PRIVATE_KEY')

In [5]:
account_obj = load_accounts(source_chain)

In [6]:
w3 = account_obj[0]['w3']

In [7]:
account = w3.eth.account.from_key(PRIVATE_KEY)
account_obj={"account": account, "w3": w3}

In [8]:
account.address

'0x38979DFdB5d8FD76FAD4E797c4660e20015C6a84'

In [9]:
gas_limit_est, min_limit_est, gas_used_values, receipts = estimate_gas_limit_from_recent_ccip_dev(w3)

In [10]:
min(gas_used_values)

200549

In [11]:
gas_limit_est

1537359

In [12]:
min_limit_est

280906

In [13]:
# raw = getattr(w3.eth, "max_priority_fee", None)
# if callable(raw):
#     tip = raw()
# elif isinstance(raw, int):
#     tip = raw
# else:
#     # fallback if your node doesn’t support it
#     tip = w3.to_wei(2, "gwei")


In [14]:
fee = w3.eth.max_priority_fee
fee

1000000

In [15]:
fees = get_dynamic_gas_fees(w3)

In [16]:
max_fee_per_gas = fees["max_fee_per_gas"]
max_fee_per_gas

1839254

In [17]:
w3.eth.gas_price

1755278

In [18]:
fees.keys()

dict_keys(['base_fee', 'max_priority_fee', 'max_fee_per_gas', 'gas_price'])

In [19]:
gas_limit_est

1537359

In [20]:
etherscan_estimate = ((gas_limit_est * max_fee_per_gas) / 1e18) * 1.25
min_etherscan_estimate = ((min_limit_est * max_fee_per_gas) / 1e18) * 1.25

In [21]:
max_gas_used_estimate = ((max(gas_used_values)*max_fee_per_gas)/ 1e18)* 1.25
min_gas_used_estimate = ((min(gas_used_values)*max_fee_per_gas)/ 1e18)* 1.25

In [22]:
gas_limit = estimate_dynamic_gas(source_chain)
gas_estimate = (gas_limit * max_fee_per_gas) / 1e18

In [23]:
gas_limit

405999

In [24]:
chain_id = w3.eth.chain_id

In [25]:
import requests

In [24]:
url = f'https://api.etherscan.io/v2/api?chainid=1&module=gastracker&action=gasoracle&apikey=H8KEKDMWJN9TX6CAZHN6IUFUV6TP8Z13CD'

r = requests.get(url)
r.json()

{'status': '1',
 'message': 'OK',
 'result': {'LastBlock': '22286609',
  'SafeGasPrice': '0.335521384',
  'ProposeGasPrice': '0.344327026',
  'FastGasPrice': '0.378759728',
  'suggestBaseFee': '0.335217161',
  'gasUsedRatio': '0.878576166666667,0.164860991337513,0.938872885015305,0.24195167442842,0.40347597898107'}}

In [26]:
print(F'etherscan_estimate: {etherscan_estimate}, min_etherscan_estimate: {min_etherscan_estimate}, dynamic_gas_estimate: {gas_estimate}, max_gas_used_estimate: {max_gas_used_estimate}, min_gas_used_estimate: {min_gas_used_estimate}')

etherscan_estimate: 3.5344921127325e-06, min_etherscan_estimate: 6.45821855155e-07, dynamic_gas_estimate: 7.46735284746e-07, max_gas_used_estimate: 3.3536750519925003e-06, min_gas_used_estimate: 4.6107568805749996e-07


In [27]:
estimate = min(etherscan_estimate,gas_estimate)
estimate

7.46735284746e-07

In [28]:
ROUTER_MAP['ethereum']

'0x80226fc0Ee2b096224EeAc085Bb9a8cba1146f7D'

In [29]:
#Approves Router to Spend Our Token If Necessary

# approve_token_if_needed(TOKEN_CONTRACTS[source_chain], 
#                             ROUTER_MAP[source_chain], 
#                             account_obj, 
#                             threshold=None)

In [30]:
receipt, links, success, message_id = send_ccip_transfer(
    to_address='0x38979DFdB5d8FD76FAD4E797c4660e20015C6a84',
    dest_chain='arbitrum',
    amount=5,
    source_chain=source_chain,
    account_obj=account_obj,
    estimate=estimate
)

2025-04-17 00:37:30,993 - INFO - CCIP messageId (pre-send): 67b82e4146ddbeb908809389d8d3bcd24e6e6687f18f6b69a671b4b5a30f1531


In [31]:
success

True

In [32]:
links

{'source_url': 'https://optimism.blockscout.com/tx/0xb74e840e04ace50ee8a3804e38abd9a42e03893555da9283d5b7f23f944a6f65',
 'ccip_url': 'https://ccip.chain.link/#/side-drawer/msg/0x67b82e4146ddbeb908809389d8d3bcd24e6e6687f18f6b69a671b4b5a30f1531'}

In [ ]:
# from web3 import Web3

# # Correct tuple-based signature
# sig = "CCIPSendRequested((bytes32,uint64,address,address,address,bytes,bytes,bytes,uint256,uint256,address))"

# # Web3 will compute the topic0 correctly
# topic0 = Web3.keccak(text=sig).hex()
# print(topic0)


003fb4702b81d9460b90fd7d986ba60b240fa8af0d1ebd672660291af318d567
